# Basic data exploration

The main tasks of this file are:
- to understand types of data contained in database files' columns
- to understand meaning of the data (for the Data Intake Report)
- to reveal possible connections between files
- to create merged dataframe based on data connections revealed for more convenient analysis

In [1]:
from datetime import datetime, timedelta

import numpy as np
print('numpy version:', np.__version__)
import pandas as pd
print('pandas version:', pd.__version__)
#import matplotlib as mpl
#print('matplotlib version:', mpl.__version__)
#import matplotlib.pyplot as plt
#import seaborn as sns
#print('seaborn version:', sns.__version__)

numpy version: 1.23.4
pandas version: 1.5.1


## Cab_Data table

In [2]:
df_cabdata = pd.read_csv('datasets/Cab_Data.csv')
df_cabdata.head()

,Transaction ID,Date of Travel,Company,City,KM Travelled,Price Charged,Cost of Trip
0,10000011,42377,Pink Cab,ATLANTA GA,30.45,370.95,313.635
1,10000012,42375,Pink Cab,ATLANTA GA,28.62,358.52,334.854
2,10000013,42371,Pink Cab,ATLANTA GA,9.04,125.20,97.632
3,10000014,42376,Pink Cab,ATLANTA GA,33.17,377.40,351.602
4,10000015,42372,Pink Cab,ATLANTA GA,8.73,114.62,97.776


In [3]:
df_cabdata.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 359392 entries, 0 to 359391
Data columns (total 7 columns):
 #   Column          Non-Null Count   Dtype  
---  ------          --------------   -----  
 0   Transaction ID  359392 non-null  int64  
 1   Date of Travel  359392 non-null  int64  
 2   Company         359392 non-null  object 
 3   City            359392 non-null  object 
 4   KM Travelled    359392 non-null  float64
 5   Price Charged   359392 non-null  float64
 6   Cost of Trip    359392 non-null  float64
dtypes: float64(3), int64(2), object(2)
memory usage: 19.2+ MB


Renaming columns for more convenient use in code

In [4]:
df_cabdata.rename(
    columns={
        'Transaction ID': 'transaction_id', 'Date of Travel': 'trip_date', 'Company': 'company', 'City': 'city', 
        'KM Travelled': 'trip_distance', 'Price Charged': 'amount_received', 'Cost of Trip': 'trip_cost'
    },
    inplace=True
)

Check columns' min and max values to detect anomalies which can be hidden NANs

In [5]:
print('Column name \tMin value \tMax value')
print('-'*45)
for column in df_cabdata.select_dtypes(exclude='object').columns:
    print('{} \t{} \t{}'.format(column, df_cabdata[column].min(), df_cabdata[column].max()))

Column name 	Min value 	Max value
---------------------------------------------
transaction_id 	10000011 	10440107
trip_date 	42371 	43465
trip_distance 	1.9 	48.0
amount_received 	15.6 	2048.03
trip_cost 	19.0 	691.2


Converting excel-like date format (number of days since starting data) to python datetime format

In [6]:
date_begin = datetime.fromisoformat('2018-12-31') - timedelta(days=int(df_cabdata['trip_date'].max()))
print('Reference date to convert `trip_date` values: {}'.format(date_begin))

Reference date to convert `trip_date` values: 1899-12-30 00:00:00


In [7]:
# check date conversion
print('Earliest trip date: {}'.format(date_begin + timedelta(days=int(df_cabdata['trip_date'].min()))))
print('Latest trip date: {}'.format(date_begin + timedelta(days=int(df_cabdata['trip_date'].max()))))

Earliest trip date: 2016-01-02 00:00:00
Latest trip date: 2018-12-31 00:00:00


Calculated period corresponds to the period mentioned in assignment description

In [8]:
df_cabdata['trip_date'] = df_cabdata['trip_date'].apply(lambda x: date_begin + timedelta(days=int(x)))
df_cabdata.head()

,transaction_id,trip_date,company,city,trip_distance,amount_received,trip_cost
0,10000011,2016-01-08,Pink Cab,ATLANTA GA,30.45,370.95,313.635
1,10000012,2016-01-06,Pink Cab,ATLANTA GA,28.62,358.52,334.854
2,10000013,2016-01-02,Pink Cab,ATLANTA GA,9.04,125.20,97.632
3,10000014,2016-01-07,Pink Cab,ATLANTA GA,33.17,377.40,351.602
4,10000015,2016-01-03,Pink Cab,ATLANTA GA,8.73,114.62,97.776


In [9]:
print('Check for `transaction_id` duplicates: max freq of `transaction_id` is {}'.format(df_cabdata['transaction_id'].value_counts().iloc[0]))

Check for `transaction_id` duplicates: max freq of `transaction_id` is 1


In [10]:
df_cabdata.to_csv('datasets/processed/trips.csv')

### Conclusion
The table contains no obvious missing / erroneous values.  
'transaction_id' column contains unique values and can be used as a key to connect to other tables.  
Assuming 'amount_receiveid' column contains data about cab companies' revenue from each trip.  
Assuming 'trip_cost' column contains data about trip's all direct costs

## Transaction_ID table

In [11]:
df_transactions = pd.read_csv('datasets/Transaction_ID.csv')
df_transactions.head()

,Transaction ID,Customer ID,Payment_Mode
0,10000011,29290,Card
1,10000012,27703,Card
2,10000013,28712,Cash
3,10000014,28020,Cash
4,10000015,27182,Card


In [12]:
df_transactions.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 440098 entries, 0 to 440097
Data columns (total 3 columns):
 #   Column          Non-Null Count   Dtype 
---  ------          --------------   ----- 
 0   Transaction ID  440098 non-null  int64 
 1   Customer ID     440098 non-null  int64 
 2   Payment_Mode    440098 non-null  object
dtypes: int64(2), object(1)
memory usage: 10.1+ MB


In [13]:
df_transactions.rename(
    columns={'Transaction ID': 'transaction_id', 'Customer ID': 'customer_id', 'Payment_Mode': 'payment_type'},
    inplace=True
)

In [14]:
for column in df_transactions.select_dtypes(exclude='object').columns:
    print('Column: {} \tmin: {} \tmax: {}'.format(column, df_transactions[column].min(), df_transactions[column].max()))

Column: transaction_id 	min: 10000011 	max: 10440108
Column: customer_id 	min: 1 	max: 60000


In [15]:
print('Check for `transaction_id` duplicates: max freq of `transaction_id` is {}'.format(df_transactions['transaction_id'].value_counts().iloc[0]))
print('Check for `customer_id` duplicates: max freq of `customer_id` is {}'.format(df_transactions['customer_id'].value_counts().iloc[0]))

Check for `transaction_id` duplicates: max freq of `transaction_id` is 1
Check for `customer_id` duplicates: max freq of `customer_id` is 54


In [16]:
df_transactions.to_csv('datasets/processed/transacts.csv')

### Conclusion
The table contains no obvious missing / erroneous values.  
The 'transaction_id' column contains unique values and can be used to link to Cab_Data table.  
The 'customer_id' column contains values that can be used as a keys to connect other tables. There are customers who had more than 1 trip

### Merging trips and transactions data

In [17]:
df_merged = df_cabdata.merge(
    df_transactions,
    how='left',
    on='transaction_id'
)

In [18]:
df_merged.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 359392 entries, 0 to 359391
Data columns (total 9 columns):
 #   Column           Non-Null Count   Dtype         
---  ------           --------------   -----         
 0   transaction_id   359392 non-null  int64         
 1   trip_date        359392 non-null  datetime64[ns]
 2   company          359392 non-null  object        
 3   city             359392 non-null  object        
 4   trip_distance    359392 non-null  float64       
 5   amount_received  359392 non-null  float64       
 6   trip_cost        359392 non-null  float64       
 7   customer_id      359392 non-null  int64         
 8   payment_type     359392 non-null  object        
dtypes: datetime64[ns](1), float64(3), int64(2), object(3)
memory usage: 27.4+ MB


In [19]:
df_merged.head()

,transaction_id,trip_date,company,city,trip_distance,amount_received,trip_cost,customer_id,payment_type
0,10000011,2016-01-08,Pink Cab,ATLANTA GA,30.45,370.95,313.635,29290,Card
1,10000012,2016-01-06,Pink Cab,ATLANTA GA,28.62,358.52,334.854,27703,Card
2,10000013,2016-01-02,Pink Cab,ATLANTA GA,9.04,125.20,97.632,28712,Cash
3,10000014,2016-01-07,Pink Cab,ATLANTA GA,33.17,377.40,351.602,28020,Cash
4,10000015,2016-01-03,Pink Cab,ATLANTA GA,8.73,114.62,97.776,27182,Card


In [20]:
df_merged.isna().sum()

transaction_id     0
trip_date          0
company            0
city               0
trip_distance      0
amount_received    0
trip_cost          0
customer_id        0
payment_type       0
dtype: int64

There are no new NAN values after merging tables. Therefore all Cab_Data rows were paired and 440098 - 359392 = 80706 transactions left.  
Let's check it

In [21]:
df_transactions.merge(df_cabdata, how='left', on='transaction_id').isna().sum()

transaction_id         0
customer_id            0
payment_type           0
trip_date          80706
company            80706
city               80706
trip_distance      80706
amount_received    80706
trip_cost          80706
dtype: int64

80 706 transactions are not paired with trip data, but we have to take them into account in our future analysis.

## Customer_ID table

In [22]:
df_customers = pd.read_csv('datasets/Customer_ID.csv')
df_customers.head()

,Customer ID,Gender,Age,Income (USD/Month)
0,29290,Male,28,10813
1,27703,Male,27,9237
2,28712,Male,53,11242
3,28020,Male,23,23327
4,27182,Male,33,8536


In [23]:
df_customers.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 49171 entries, 0 to 49170
Data columns (total 4 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   Customer ID         49171 non-null  int64 
 1   Gender              49171 non-null  object
 2   Age                 49171 non-null  int64 
 3   Income (USD/Month)  49171 non-null  int64 
dtypes: int64(3), object(1)
memory usage: 1.5+ MB


In [24]:
df_customers.rename(
    columns={'Customer ID': 'customer_id', 'Gender': 'gender', 'Age': 'age', 'Income (USD/Month)': 'monthly_income'},
    inplace=True
)

In [25]:
for column in df_customers.select_dtypes(exclude='object').columns:
    print('Column: {} \tmin: {} \tmax: {}'.format(column, df_customers[column].min(), df_customers[column].max()))

Column: customer_id 	min: 1 	max: 60000
Column: age 	min: 18 	max: 65
Column: monthly_income 	min: 2000 	max: 35000


In [26]:
print('Check for `customer_id` duplicates: max freq of `customer_id` is {}'.format(df_customers['customer_id'].value_counts().iloc[0]))

Check for `customer_id` duplicates: max freq of `customer_id` is 1


'customer_id' column contains unique values

In [27]:
# check if all customers are presented in transactions by customer_id
cnt = 0
cust_ids = pd.unique(df_customers['customer_id'])
for cid in pd.unique(df_transactions['customer_id']):
    if cid not in cust_ids:
        cnt += 1
print('Number of customers not mentioned in Transaction_ID table: {}'.format(cnt))

Number of customers not mentioned in Transaction_ID table: 0


In [28]:
# opposite check if all customers are presented in customers table by customer_id
cnt = 0
cust_ids = pd.unique(df_transactions['customer_id'])
for cid in pd.unique(df_customers['customer_id']):
    if cid not in cust_ids:
        cnt += 1
print('Number of customers from Transaction_ID table not mentioned in Customer_ID table: {}'.format(cnt))

Number of customers from Transaction_ID table not mentioned in Customer_ID table: 0


We have full correspondence of customers with transactions data

### Some customers table logical checks

#### Important note! Youngest passenger(s) are only 18 y.o. while credit cards in US are issued usually if person's age is more than 20 y.o.  

In [29]:
print('Average income of passengers younger than 21 y.o. is {:.2f} USD'.format(df_customers[df_customers['age'] < 21]['monthly_income'].mean()))

Average income of passengers younger than 21 y.o. is 14938.05 USD


#### Important note! Average monthly income in USA in 2021 was $5870 (https://www.worlddata.info/average-income.php)

In [30]:
df_customers.to_csv('datasets/processed/customers.csv')

### Conclusion
The table contains no obvious missing values.  
Some values are suspicious in terms of age <-> payment type (card) and age <-> monthly income.  
The 'customer_id' column contains unique values that can be used as a keys to connect to Transaction_ID table.

### Merging synthetic dataset with customers data

In [31]:
df_merged = df_merged.merge(
    right=df_customers,
    how='left',
    on='customer_id'
)
df_merged.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 359392 entries, 0 to 359391
Data columns (total 12 columns):
 #   Column           Non-Null Count   Dtype         
---  ------           --------------   -----         
 0   transaction_id   359392 non-null  int64         
 1   trip_date        359392 non-null  datetime64[ns]
 2   company          359392 non-null  object        
 3   city             359392 non-null  object        
 4   trip_distance    359392 non-null  float64       
 5   amount_received  359392 non-null  float64       
 6   trip_cost        359392 non-null  float64       
 7   customer_id      359392 non-null  int64         
 8   payment_type     359392 non-null  object        
 9   gender           359392 non-null  object        
 10  age              359392 non-null  int64         
 11  monthly_income   359392 non-null  int64         
dtypes: datetime64[ns](1), float64(3), int64(4), object(4)
memory usage: 35.6+ MB


No new NAN data after merging

In [32]:
df_merged.head()

,transaction_id,trip_date,company,city,trip_distance,amount_received,trip_cost,customer_id,payment_type,gender,age,monthly_income
0,10000011,2016-01-08,Pink Cab,ATLANTA GA,30.45,370.95,313.635,29290,Card,Male,28,10813
1,10000012,2016-01-06,Pink Cab,ATLANTA GA,28.62,358.52,334.854,27703,Card,Male,27,9237
2,10000013,2016-01-02,Pink Cab,ATLANTA GA,9.04,125.20,97.632,28712,Cash,Male,53,11242
3,10000014,2016-01-07,Pink Cab,ATLANTA GA,33.17,377.40,351.602,28020,Cash,Male,23,23327
4,10000015,2016-01-03,Pink Cab,ATLANTA GA,8.73,114.62,97.776,27182,Card,Male,33,8536


### Saving new dataset to file for further analysis (EDA)

In [33]:
df_merged.to_csv('datasets/processed/merged_trips_transacts_cust.csv')

### Creating new dataset with merged transactions and customers

In [34]:
df_merged = df_transactions.merge(df_customers, how='right', on='customer_id')
df_merged.head()

,transaction_id,customer_id,payment_type,gender,age,monthly_income
0,10000011,29290,Card,Male,28,10813
1,10351127,29290,Cash,Male,28,10813
2,10412921,29290,Card,Male,28,10813
3,10000012,27703,Card,Male,27,9237
4,10320494,27703,Card,Male,27,9237


In [35]:
df_merged.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 440098 entries, 0 to 440097
Data columns (total 6 columns):
 #   Column          Non-Null Count   Dtype 
---  ------          --------------   ----- 
 0   transaction_id  440098 non-null  int64 
 1   customer_id     440098 non-null  int64 
 2   payment_type    440098 non-null  object
 3   gender          440098 non-null  object
 4   age             440098 non-null  int64 
 5   monthly_income  440098 non-null  int64 
dtypes: int64(4), object(2)
memory usage: 23.5+ MB


In [36]:
df_merged.to_csv('datasets/processed/merged_transacts_cust.csv')

## City table

In [2]:
df_city = pd.read_csv('datasets/City.csv')
df_city

,City,Population,Users
0,NEW YORK NY,"8,405,837","302,149"
1,CHICAGO IL,"1,955,130","164,468"
2,LOS ANGELES CA,"1,595,037","144,132"
3,MIAMI FL,"1,339,155","17,675"
4,SILICON VALLEY,"1,177,609","27,247"
5,ORANGE COUNTY,"1,030,185","12,994"
6,SAN DIEGO CA,"959,307","69,995"
7,PHOENIX AZ,"943,999","6,133"
8,DALLAS TX,"942,908","22,157"
9,ATLANTA GA,"814,885","24,701"


In [3]:
df_city.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20 entries, 0 to 19
Data columns (total 3 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   City        20 non-null     object
 1   Population  20 non-null     object
 2   Users       20 non-null     object
dtypes: object(3)
memory usage: 608.0+ bytes


All columns in table are of 'object' type while 'Population' and 'Users' columns contain numeric data. Have to convert them

In [4]:
df_city['num_users'] = df_city['Users'].apply(lambda x: int(x.replace(',', '')))
df_city['population'] = df_city['Population'].apply(lambda x: int(x.replace(',', '')))
df_city.drop(columns=['Population', 'Users'], inplace=True)
df_city.rename(columns={'City': 'city'}, inplace=True)
df_city.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20 entries, 0 to 19
Data columns (total 3 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   city        20 non-null     object
 1   num_users   20 non-null     int64 
 2   population  20 non-null     int64 
dtypes: int64(2), object(1)
memory usage: 608.0+ bytes


Let's check if all the cities are in Cab_Data table

In [5]:
cities_used = list(df_cabdata['city'].value_counts().index)
for city in df_city['city'].value_counts().index:
    if city in cities_used:
        print(f'City: {city}: OK')
    else:
        print(f'City: {city} NOT is in df_cabdata stat')

NameError: name 'df_cabdata' is not defined

In [6]:
df_city.to_csv('datasets/processed/cities.csv')

### Conclusion
The table contains no obvious missing values.  
The table contains one city which has no correspondent records in Cab_Data table.  
The meaning of the 'Population' column not suite the official US Census Bureau data (see https://www.moderncities.com/article/2017-jun-top-100-us-cities-ranked-by-2016-population).  
The meaning of 'Users' column is unclear. Assume this is number of potential customers of cab companies (estimated number of people that use any cab company services) got from outer source (marketing department, agencies, etc.)

## OVERALL CONCLUSION

We have analyzed the content of files containing data.  
We have made some checks about data consistency.  
We have transformed some data to more convenient formats.  
We have created new datasets by partially merging data from several tables.  
(!) We have found some points in which data is not suited to web sources.  